In [4]:
import requests

def ask_mistral(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral",  # Name des installierten Modells
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json()["response"]

# Beispiel:
antwort = ask_mistral("Erkläre Quantum Computing in 3 Sätzen:")
print(antwort)


1. Quantum Computing ist die Art und Weise, Algorithmen auf Quantensystemen auszuführen statt klassischen Hardware, um problematische Berechnungen schneller zu lösen als mit konventionellen Rechenmethoden möglich wäre.

2. Das Grundprinzip von Quantum Computing ist das Quantenparalleling: Ein Quantenzustand kann zugleich mehrere Werte besitzen, was dazu führt, dass ein Quantenrechner viele Berechnungen simultan ausführt und so die Leistungsfähigkeit des Rechners erheblich verbessert.

3. Die große Potenziale von Quantum Computing liegt darin, dass sie Probleme löst, die für klassische Rechenmethoden unsolvable sind oder für die eine zu lange Lösungszeit benötigt wird. Beispiele hierfür sind die Faktorisierung großer Zahlen und das Lösen von NP-vollständigen Problemen.


In [2]:
import requests

response = requests.post(
    "http://localhost:11434/api/generate",
    json={"model": "mistral", "prompt": "Was ist Data Science?"},
)

# Ausgabe der gesamten Antwort zur Überprüfung
print(response.text)


{"model":"mistral","created_at":"2025-03-30T11:25:37.0744006Z","response":" Data","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:39.9498019Z","response":" Science","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:41.9949553Z","response":" ist","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:42.5209975Z","response":" eine","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:42.9803663Z","response":" mult","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:43.472495Z","response":"id","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:43.9056848Z","response":"is","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:44.3243327Z","response":"zi","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:44.753716Z","response":"pl","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:45.1723388Z","response":"in","done":false}
{"model":"mistral","created_at":"2025-03-30T11:25:45.61